In [617]:
import pandas as pd
import numpy as np

In [618]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [619]:
class SPredictionDataset(Dataset):
    """S Prediction dataset."""

    def __init__(self, csv_file):
        dtype = {'Point-ID': np.int32, 'Hole-ID': str, 'Sample': str, 'From': np.float64, 'To': np.float64, 
         'M': np.float64, 'Au': np.float64, 'Ag': np.float64, 'X': np.float64, 'Y': np.float64, 'Z': np.float64, 
         'Block': str, 'Date': str, 'S': np.float64, 'Fe': np.float64, 'Cu': np.float64, 
         'Zn': np.float64, 'As': np.float64, 'Cd': np.float64, 'Sb': np.float64, 'Pb': np.float64, 'Bi': np.float64, 
         'Au_eq': np.float64, 'Pb_eq': np.float64, 'Au_Cut': np.float64, 'Ag_Cut': np.float64, 'Pb_Cut': np.float64, 
         'Zn_Cut': np.float64, 'Cu_Cut': np.float64, 'As_Cut': np.float64, 'Sb_Cut': np.float64, 
         'RT_2021': str, 'Cd_Cut': np.float64, 'Bi_Cut': np.float64, 'Code': str, 
        'RT_2021_Бедные': str}
        df = pd.read_csv(csv_file, sep=';', header='infer', na_values='na', 
                     keep_default_na=True, na_filter=True, verbose=False, decimal=',', dtype=dtype,
                     usecols=['Au', 'Ag', 'S', 'Fe', 'Cu', 'Zn', 'As', 'Cd', 'Sb', 'Pb', 'Bi'])
        
        df_partial = df[['Ag', 'S', 'Fe', 'Cu', 'Sb', 'Pb', 'Bi']]
        #print ('df_partial=', df_partial)
        df_notnull = df_partial.notnull()
        #print ('df_notnull=', df_notnull)
        rows = df_notnull.all(1)
        #print ('rows=', rows)
        self.df = df[rows]
        #print ('self.df=', self.df)

        self.target = 'S'

        # Save target and predictors
        self.X = self.df.drop(self.target, axis=1)
        #print ('self.X=', self.X)
        self.y = self.df[self.target]
        #print ('self.y=', self.y)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return [self.X.iloc[idx].values, self.y.iloc[idx]]

In [620]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

device = get_device()

In [621]:
class FullyConnected(nn.Module):
    def __init__(self, sizes, dropout=[], bn=False, activation_fn=nn.Tanh(), flatten=False, 
                 last_fn=None, first_fn=None, device='cpu'):
        super(FullyConnected, self).__init__()
        layers = []
        self.flatten = flatten
        if first_fn is not None:
            layers.append(first_fn)
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            layers.append(activation_fn)
            if bn:
                layers.append(nn.BatchNorm1d(sizes[i+1]))
            if dropout:
                layers.append(nn.Dropout(dropout[i]))
        else: 
            layers.append(nn.Linear(sizes[-2], sizes[-1]))
        if last_fn is not None:
            layers.append(last_fn)
        self.model = nn.Sequential(*layers)
        self.to(device)
        
    def forward(self, x, y=None):
        if self.flatten:
            x = x.view(x.shape[0], -1)
        if y is not None:
            x = torch.cat([x, y], dim=1)
        return self.model(x)

In [622]:
class Model(nn.Module):
    def __init__(self, dims, dropout=[], bn=False, activation_fn=nn.ReLU(), last_fn=nn.ReLU(), lr=0.001, lr_decay=0.0001, weight_decay=0.01):
        super(Model, self).__init__()
        self.fc = FullyConnected(dims, dropout=dropout, bn=bn, activation_fn=activation_fn, last_fn=last_fn, flatten=False, device=device)
        self.optimizer = optim.Adagrad(self.parameters(), lr=lr, lr_decay=lr_decay, 
                                       weight_decay=weight_decay, initial_accumulator_value=0, eps=1e-10)
        self.loss = torch.nn.MSELoss()
    
    def forward(self, x):
        return self.fc(x)

In [623]:
def train(mode, train_loader):
    print ('Training on device {}'.format(device))
    
    epochs = 100
    train_size = len(train_loader)
    
    for epoch in range(epochs):
        for batch_idx, (X_train, y_train) in enumerate(train_loader):
            #print ("X_train=", X_train)
            model.optimizer.zero_grad()
            X_train_tensor = X_train.float().to(device)
            y_train_tensor = y_train.float().to(device)
            y_pred = model(X_train_tensor)
            y_pred = y_pred.view(-1)
            #print ('y_pred.shape',y_pred.shape)
            #print ('y_train_tensor.shape',y_train_tensor.shape)
            loss = model.loss(y_pred, y_train_tensor)
            loss.backward()

            model.optimizer.step()

            if batch_idx % 100 == 0:
                line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                    epoch, batch_idx, train_size, 100. * batch_idx / train_size)
                losses = '{:.4f}'.format(loss.item())
                print(line + losses)

In [624]:
def test(model, test_loader):
    dataiter = iter(test_loader)
    X_test, y_test = dataiter.next()

    X_test_tensor = X_test.float().to(device)
    y_test_tensor = y_test.float().to(device)
        
    y_pred = model(X_test_tensor)
    y_pred = y_pred.view(-1)
        
    loss = model.loss(y_pred, y_test_tensor).item()
        
    print ("Test loss={:.4f}".format(loss))
    print ("y_pred=", y_pred)
    print ("y_test=", y_test)

In [625]:
# Load dataset
dataset = SPredictionDataset('DH_ASSAY_General_2021.csv')

#print ('dataset.X.shape[1]=', dataset.X.shape[1])
# Define model
layers = dataset.X.shape[1], 1000, 500, 250, 1
model = Model(layers, dropout=[0.001, 0.01, 0.01], bn=True, activation_fn=nn.ReLU(inplace=True),
              last_fn=nn.ReLU(inplace=True), 
              lr=.001, 
              weight_decay=0.0001)
print (model)

# Split into training and test
train_size = int(0.8 * len(dataset))
print ('train_size=', train_size)
test_size = len(dataset) - train_size
print ('test_size=', test_size)
train_set, test_set = random_split(dataset, [train_size, test_size])

# Dataloaders
train_loader = DataLoader(train_set, batch_size=200, shuffle=True)
test_loader = DataLoader(test_set, batch_size=200, shuffle=False)

Model(
  (fc): FullyConnected(
    (model): Sequential(
      (0): Linear(in_features=10, out_features=1000, bias=True)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.001, inplace=False)
      (4): Linear(in_features=1000, out_features=500, bias=True)
      (5): ReLU(inplace=True)
      (6): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): Dropout(p=0.01, inplace=False)
      (8): Linear(in_features=500, out_features=250, bias=True)
      (9): ReLU(inplace=True)
      (10): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): Dropout(p=0.01, inplace=False)
      (12): Linear(in_features=250, out_features=1, bias=True)
      (13): ReLU(inplace=True)
    )
  )
  (loss): MSELoss()
)
train_size= 82701
test_size= 20676


In [626]:
train(model, train_loader)
test(model, test_loader)

Training on device cuda:0
Train Epoch: 0 [0/414 (0%)]	Losses 8.2205
Train Epoch: 0 [100/414 (24%)]	Losses 0.7327
Train Epoch: 0 [200/414 (48%)]	Losses 1.5559
Train Epoch: 0 [300/414 (72%)]	Losses 14.6150
Train Epoch: 0 [400/414 (97%)]	Losses 16.7970
Train Epoch: 1 [0/414 (0%)]	Losses 2.6718
Train Epoch: 1 [100/414 (24%)]	Losses 1.3571
Train Epoch: 1 [200/414 (48%)]	Losses 1.3325
Train Epoch: 1 [300/414 (72%)]	Losses 1.6373
Train Epoch: 1 [400/414 (97%)]	Losses 0.7128
Train Epoch: 2 [0/414 (0%)]	Losses 0.7749
Train Epoch: 2 [100/414 (24%)]	Losses 0.7392
Train Epoch: 2 [200/414 (48%)]	Losses 1.1895
Train Epoch: 2 [300/414 (72%)]	Losses 0.6846
Train Epoch: 2 [400/414 (97%)]	Losses 0.7686
Train Epoch: 3 [0/414 (0%)]	Losses 0.9260
Train Epoch: 3 [100/414 (24%)]	Losses 0.7797
Train Epoch: 3 [200/414 (48%)]	Losses 1.2093
Train Epoch: 3 [300/414 (72%)]	Losses 0.9574
Train Epoch: 3 [400/414 (97%)]	Losses 0.8229
Train Epoch: 4 [0/414 (0%)]	Losses 6.8793
Train Epoch: 4 [100/414 (24%)]	Losses 0.96

Train Epoch: 36 [100/414 (24%)]	Losses 1.7879
Train Epoch: 36 [200/414 (48%)]	Losses 1.4845
Train Epoch: 36 [300/414 (72%)]	Losses 1.5331
Train Epoch: 36 [400/414 (97%)]	Losses 3.4575
Train Epoch: 37 [0/414 (0%)]	Losses 0.6640
Train Epoch: 37 [100/414 (24%)]	Losses 0.5358
Train Epoch: 37 [200/414 (48%)]	Losses 0.5935
Train Epoch: 37 [300/414 (72%)]	Losses 0.7249
Train Epoch: 37 [400/414 (97%)]	Losses 0.6962
Train Epoch: 38 [0/414 (0%)]	Losses 4.7442
Train Epoch: 38 [100/414 (24%)]	Losses 0.8398
Train Epoch: 38 [200/414 (48%)]	Losses 0.5186
Train Epoch: 38 [300/414 (72%)]	Losses 0.7789
Train Epoch: 38 [400/414 (97%)]	Losses 0.6595
Train Epoch: 39 [0/414 (0%)]	Losses 0.5220
Train Epoch: 39 [100/414 (24%)]	Losses 0.8637
Train Epoch: 39 [200/414 (48%)]	Losses 0.7581
Train Epoch: 39 [300/414 (72%)]	Losses 0.5973
Train Epoch: 39 [400/414 (97%)]	Losses 0.7005
Train Epoch: 40 [0/414 (0%)]	Losses 1.8514
Train Epoch: 40 [100/414 (24%)]	Losses 0.6345
Train Epoch: 40 [200/414 (48%)]	Losses 0.6885


Train Epoch: 72 [200/414 (48%)]	Losses 0.9081
Train Epoch: 72 [300/414 (72%)]	Losses 1.1675
Train Epoch: 72 [400/414 (97%)]	Losses 3.5415
Train Epoch: 73 [0/414 (0%)]	Losses 0.7004
Train Epoch: 73 [100/414 (24%)]	Losses 0.5830
Train Epoch: 73 [200/414 (48%)]	Losses 0.5345
Train Epoch: 73 [300/414 (72%)]	Losses 0.4886
Train Epoch: 73 [400/414 (97%)]	Losses 0.4608
Train Epoch: 74 [0/414 (0%)]	Losses 0.7954
Train Epoch: 74 [100/414 (24%)]	Losses 0.8085
Train Epoch: 74 [200/414 (48%)]	Losses 0.5170
Train Epoch: 74 [300/414 (72%)]	Losses 0.4479
Train Epoch: 74 [400/414 (97%)]	Losses 0.7706
Train Epoch: 75 [0/414 (0%)]	Losses 0.6464
Train Epoch: 75 [100/414 (24%)]	Losses 0.4496
Train Epoch: 75 [200/414 (48%)]	Losses 1.2528
Train Epoch: 75 [300/414 (72%)]	Losses 2.5849
Train Epoch: 75 [400/414 (97%)]	Losses 0.8965
Train Epoch: 76 [0/414 (0%)]	Losses 0.7136
Train Epoch: 76 [100/414 (24%)]	Losses 0.9715
Train Epoch: 76 [200/414 (48%)]	Losses 0.5225
Train Epoch: 76 [300/414 (72%)]	Losses 0.5230


In [ ]:
with torch.no_grad():
    model.eval()
    output = model.forward(test_cats,test_conts).cuda()
output